In [1]:
from FNN import FNN
from Layer import Layer
from Neuron import Neuron
import numpy as np
from ActivFunctions import  *
from InitFunctions import  *
from SuppFunctions import  *
from ErrorClasses import *

Neuron Class functionality testing

In [2]:
#zero initialization
neuron = Neuron(2,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [3]:
#random initialization
neuron = Neuron(2,identity,method_ini = "RandomNor", random_mean = -1000.0, random_std = 10.0)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [4]:
#initialization from list
vector = [1.0,2.0,3.5]
neuron = Neuron(vector,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [5]:
#initialization from vector
vector = np.array([1.0,2.0,3.5])
neuron = Neuron(vector,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [6]:
#initialization from row vector(2D array)
vector = np.array([1.0,2.0,3.5])
vector_row = vector[np.newaxis,:]
neuron = Neuron(vector_row,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [7]:
#initialization from column vector(2D array)
vector = np.array([1.0,2.0,3.5])
vector_column = vector[:,np.newaxis]
neuron = Neuron(vector_column,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [8]:
#error messages testing
print("Test for: no function given as activation function")
try:
    vector = np.array([1.0,2.0,3.5])
    errorNeuron = Neuron(vector,1)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not vector given for weights initialization (2 dim)")
try:
    vector = np.array([[1.0,2.0,3.5],[1.0,2.0,3.5]])
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not vector given for weights initialization (higher than 2 dim)")
try:
    vector = np.array([[[1.0,2.0,3.5],[1.0,2.0,3.5]],[[1.0,2.0,3.5],[1.0,2.0,3.5]]])
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong weights given for initialization")
try:
    vector = np.array(["1.0","2.0","3.5"])
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type given in list for weight initialization")
try:
    vector = ["as","2.0","3.5"]
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - inproper dimension of input")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="RandomNor")
    inputTest = np.array([[[1.0,1.0],[1.0,1.0]],[[1.0,1.0],[1.0,1.0]]]) 
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - wrong value types in input: they are not rational numbers")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="RandomNor")
    inputTest = np.array([1.0 + 3j,1.0]) 
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given variable is of not supported type for list")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="RandomNor")
    inputTest = ["a","1.0"]
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given variable is of not supported type")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="RandomNor")
    inputTest = "a"
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given input is too small to match neuron dimensions")
try:
    #wrong input (too small)
    vector = np.array([1.0,2.0,3.5])
    neuronTest = Neuron(vector,identity)
    inputTestErrorSmall = np.array([1.0])
    neuronTest.forward(inputTestErrorSmall)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given input is too small to match neuron dimensions")
try:
    #wrong input (too big)
    vector = np.array([1.0,2.0,3.5])
    neuronTest = Neuron(vector,identity)
    inputTestErrorBig = np.array([[1.0],[1.0],[1.0],[1.0]])
    neuronTest.forward(inputTestErrorBig)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")

Test for: no function given as activation function
Correct errors was caught. Error: Given input is not supported by implementation for activation function initialization. Given variable is not a function and thus can not be used as activation function.
Test for: not vector given for weights initialization (2 dim)
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given variable is not an vector (not column or row vector in 2 dim), so it is unsuitable for neuron weight initialization.
Test for: not vector given for weights initialization (higher than 2 dim)
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given variable is not an vector (due to more than 2 dim), so it is unsuitable for neuron weight initialization.
Test for: wrong weights given for initialization
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization.

Layer class implementation testing

In [9]:
#initialization by number of dim in vector
dim_layer = np.array([4,5])
testLayer = Layer(dim_layer,identity,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [10]:
#initialization by number of dim in list
dim_layer = [4,5]
testLayer = Layer(dim_layer,identity,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [11]:
#initialization by giving weight array
weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
testLayer = Layer(weights,identity,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [12]:
#initialization by giving weight array and function (single) from list
weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
testLayer = Layer(weights,[identity],method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [13]:
#initialization by giving weight array and identity function (mutliple same) from list
weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
activation_functions = [identity,identity,identity]
testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [14]:
#initialization by giving weight array and identity function (mutliple different) from list
weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
activation_functions = [sigmoid,relu,leaky_relu]
testLayer = Layer(weights,activation_functions)
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [15]:
#error messages testing
print("Test for: wrong var type (not number) given in list for weight initialization")
try:
    vector = ["as","2.0"]
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type given (number, but no int) in list for weight initialization")
try:
    vector = [1,2.3]
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type (not number) given in 1D array for weight initialization")
try:
    vector = np.array(["as","2.0"])
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type given (number, but no int) in 1D array for weight initialization")
try:
    vector = np.array([1,2.3])
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong dimensions of given 1D array for weight initialization")
try:
    vector = np.array([1,2,1,1,2])
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not rational numbers given in 2D array for weight initialization")
try:
    weights = np.array([[1.0 + 4j,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not enough information given in 2D array for weight initialization")
try:
    weights = np.array([[]])
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: bigger than 2D was given for array for weight initialization")
try:
    weights = np.array([[[2]]])
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not supported data type given for array for weight initialization")
try:
    weights = "hehe"
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not supported data type given for array for weight initialization")
try:
    weights = "hehe"
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not compatible amount of activation functions given to the amount of neurons for activation function initialization")
try:
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,relu,leaky_relu,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not activation function given (inside list) for activation function initialization")
try:
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,1,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not activation function given (standalone) for activation function initialization")
try:
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = 1
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, too much dimensions in input")
try:
    #Input data for input propagation (forward pass)
    inputTestArray = np.array([[[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0]],[[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0]]])
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestArray)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, complex input")
try:
    #Input data for input propagation (forward pass)
    inputTestVect = np.array([[1.0 + 3j],[1.0],[1.0],[1.0],[1.0]])
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestVect)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, wrong data type of input")
try:
    #Input data for input propagation (forward pass)
    inputTestVect = "sda"
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestVect)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, wrong data type of input in list")
try:
    #Input data for input propagation (forward pass)
    inputTestList =["a",1.0,1.0,1.0,1.0]
    inputTestArray = np.array([[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0]])
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestList)
    testLayer.forward(inputTestArray)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, input does not match layer neurons")
try:
    #Input data for input propagation (forward pass)
    inputTestList = [1.0,1.0,1.0,1.0,1.0,1.0,1.0]
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestList)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")

Test for: wrong var type (not number) given in list for weight initialization
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given values are not integers and thus can not represent of layer weights matrix.
Test for: wrong var type given (number, but no int) in list for weight initialization
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given values are not integers and thus can not represent of layer weights matrix.
Test for: wrong var type (not number) given in 1D array for weight initialization
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given values are not integers and thus can not represent of layer weights matrix.
Test for: wrong var type given (number, but no int) in 1D array for weight initialization
Correct errors was caught. Error: Given input is not supported by implementation for weight

FNN class implementation testing

In [16]:
#initialization by number of dim in vector(list)
dim_layer = [5,5,3,2,1]
activ_func = [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
print(testNet.weights_list[0])
print(testNet.weights_list[1])
print(testNet.weights_list[2])
print(testNet.weights_list[3])
print(testNet.activ_functions_list_list[0])
print(testNet.activ_functions_list_list[1])
print(testNet.activ_functions_list_list[2])
print(testNet.activ_functions_list_list[3])
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

[[ 1.33364527  0.44291911 -0.75092619  1.10670345 -1.2077891  -0.10141314]
 [ 1.00774355 -0.0538972  -0.97181407 -0.80297793  0.67920202  0.46027571]
 [-0.06448236  1.40476249 -0.27646015  1.11922454  0.15028142  0.54246289]
 [ 0.00883645  0.12087688  1.15872418  0.27110307  0.57679682 -1.17195185]
 [ 0.4060773  -0.77232796  0.13690842 -0.49304674 -0.18084542  0.73728327]]
[[-1.27497098  0.04669725  0.33392552 -1.74958122 -0.37847487 -2.01253947]
 [-1.14424047  0.39063725 -0.60219442 -0.58631938  0.6832578  -1.07979108]
 [-0.27177721 -1.66948544 -0.44437837 -0.69964313 -0.0587253   0.03453719]]
[[ 0.54294564 -1.50893788  1.36386847  0.30065018]
 [ 0.66830714  0.21877324  0.23011319 -0.22077466]]
[[-1.0911734   1.28788291  0.10407482]]
[<function relu at 0x00000289F7B537E0>, <function relu at 0x00000289F7B537E0>, <function relu at 0x00000289F7B537E0>, <function relu at 0x00000289F7B537E0>, <function relu at 0x00000289F7B537E0>]
[<function relu at 0x00000289F7B537E0>, <function relu at 0

True

In [17]:
#initialization by number of dim in vector(1D array)
dim_layer = np.array([5,5,3,2,1])
activ_func = [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [18]:
#dummy network ini
dim_layer = np.array([5,5,3,2,1])
activ_func = [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#initialization by ready weight array
dim_layer = testNet.weights_list
print(dim_layer)
activ_func = [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

[array([[ 0.66047625, -0.33276176,  0.40488953,  0.89390386, -0.34723361,
         0.34087916],
       [-0.83422565,  1.35907814,  0.84067946, -0.65450944, -1.06825306,
        -0.18477486],
       [-0.12659128, -1.0018063 ,  0.19347669,  2.07161662, -0.60319664,
         1.8710192 ],
       [-2.30987484,  0.36369003,  1.41969981, -1.43353451,  1.55535559,
        -1.94625692],
       [-0.50848118, -0.39356117,  0.80879137, -2.00050033,  0.63254133,
        -0.45344149]]), array([[ 1.71568589, -0.47266727,  0.33622452,  0.41760887,  1.22272247,
        -1.76765824],
       [-1.42385156,  0.06365852, -1.19952379,  0.22832264,  1.21981314,
         0.28637416],
       [-0.12301628, -1.76464243, -2.27208323,  1.32439937, -0.6484877 ,
         1.40878617]]), array([[ 0.01365947, -0.14194297,  0.95801234,  0.01783131],
       [ 0.19868577, -0.31155288, -0.52614998,  3.20390385]]), array([[ 0.7152957 ,  0.48243009, -0.21271528]])]


True

In [19]:
#initialization by number of dim in vector(1D array) and 1 activation function(as value)
dim_layer = np.array([5,5,3,2,1])
activ_func = identity
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [20]:
#initialization by number of dim in vector(1D array) and 1 activation function(as list)
dim_layer = np.array([5,5,3,2,1])
activ_func = [identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [21]:
#initialization by number of dim in vector(1D array) and 2 activation functions
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [22]:
#initialization by number of dim in vector(1D array) and activation function for each layer
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,sigmoid,leaky_relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [23]:
#initialization by number of dim in vector(1D array) and activation function for each neuron
dim_layer = np.array([5,5,3,2,1])
l1 = [relu,sigmoid,sigmoid,leaky_relu,identity]
l2 = [relu,sigmoid,sigmoid]
l3 = [relu,sigmoid]
l4 = [identity]
activ_func =  [l1,l2,l3,l4]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [24]:
#initialization by number of dim in vector(1D array) and 2 activation functions with Zero weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="Zero")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [25]:
#initialization by number of dim in vector(1D array) and 2 activation functions with random uniform weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomUni")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [26]:
#initialization by number of dim in vector(1D array) and 2 activation functions with random normalized weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [27]:
#initialization by number of dim in vector(1D array) and 2 activation functions with Xavier uniform weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="XavUni")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [28]:
#initialization by number of dim in vector(1D array) and 2 activation functions with Xavier normalized weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="XavNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [29]:
#initialization by number of dim in vector(1D array) and 2 activation functions with He uniform weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="HeUni")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [30]:
#initialization by number of dim in vector(1D array) and 2 activation functions with He normalized weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="HeNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [31]:
#initialization by number of dim in vector(1D array) and 2 activation functions with He normalized weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="HeNor")
#decomposing network
layers_list = testNet.decomposeIntoLayers()
neurons_list_list = testNet.decomposeIntoNeurons()
#checking the results
print(layers_list)
print(neurons_list_list)

[<Layer.Layer object at 0x00000289F7BF8650>, <Layer.Layer object at 0x00000289F7BF94D0>, <Layer.Layer object at 0x00000289F7BFA390>, <Layer.Layer object at 0x00000289F7BFB150>]
[[<Neuron.Neuron object at 0x00000289F7BF8790>, <Neuron.Neuron object at 0x00000289F7BF83D0>, <Neuron.Neuron object at 0x00000289F7BF8E50>, <Neuron.Neuron object at 0x00000289F7BF9850>, <Neuron.Neuron object at 0x00000289F7BF9E50>], [<Neuron.Neuron object at 0x00000289F7BF9410>, <Neuron.Neuron object at 0x00000289F7BFA410>, <Neuron.Neuron object at 0x00000289F7BFABD0>], [<Neuron.Neuron object at 0x00000289F7BFA450>, <Neuron.Neuron object at 0x00000289F7BFA510>], [<Neuron.Neuron object at 0x00000289F7BFA7D0>]]


In [32]:
#error messages testing
print("Test for: not supported input for weights")
try:
    dim_layer = "[5,5,3,2,1]"
    testNet = FNN(dim_layer,identity,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: array of not supported dimension")
try:
    dim_layer = np.array([[5,5,3,2,1],[5,5,3,2,1]])
    testNet = FNN(dim_layer,identity,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: given numbers are not integers")
try:
    dim_layer = np.array([5,5,3.1,2,1])
    testNet = FNN(dim_layer,identity,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not enough numbers given. Minimally required input and output layers cannot be created")
try:
    dim_layer = np.array([5])
    testNet = FNN(dim_layer,identity,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")

Test for: not supported input for weights
Correct errors was caught. Error:  Given input is not supported by implementation for weights initialization. Not supported data type given.
Test for: array of not supported dimension
Correct errors was caught. Error:  Given array is of size not supported by implementation. Supported dimensions: 1.
Test for: given numbers are not integers
Correct errors was caught. Error:  Given input is not supported by implementation for weights initialization. Given values in vector must be integers to properly represent number of neurons in layers
Test for: not enough numbers given. Minimally required input and output layers cannot be created
Correct errors was caught. Error:  Given input is not supported by implementation for weights initialization. At least two numbers are needed for FNN creation: FNN needs at least input and output layer (single layer network).


Reload Modules


In [33]:
import importlib
import sys
import numpy as np

# Reload order:
modules_in_order = [
    "ActivFunctions",
    "SuppFunctions",
    "InitFunctions",
    "Layer",
    "FNN",
    "TrainingFunctions"
]

for m in modules_in_order:
    if m in sys.modules:
        importlib.reload(sys.modules[m])
    else:
        globals()[m] = importlib.import_module(m)

print("All modules reloaded successfully.")


All modules reloaded successfully.


Imports

In [34]:
from FNN import FNN
from TrainingFunctions import backwards
from LossFunctions import MeanSquaredErrorDerivative

# Activations
from ActivFunctions import identity, sigmoid, tanh, relu, leaky_relu
import numpy as np


In [35]:
print("=== Test 1: Single-Layer Network Gradient ===")

dim_layer = np.array([4, 5])   # 4 inputs -> 5 outputs
testNet = FNN(dim_layer, identity, method_ini="RandomNor")

inputTestVect = np.ones((4, 1))
targetVect = 0.5 * np.ones((5, 1))

out = testNet.forward(inputTestVect)
grad = testNet.backward(out[0],out[1],targetVect,MeanSquaredErrorDerivative)

#grads = backwards(testNet, inputTestVect, targetVect, MeanSquaredErrorDerivative)

print("\nGradient matrix for single layer:")
print(grad[0])
print(grad)


=== Test 1: Single-Layer Network Gradient ===

Gradient matrix for single layer:
[[-0.15235355 -0.15235355 -0.15235355 -0.15235355 -0.15235355]
 [-3.17578605 -3.17578605 -3.17578605 -3.17578605 -3.17578605]
 [-2.07655263 -2.07655263 -2.07655263 -2.07655263 -2.07655263]
 [-3.39723958 -3.39723958 -3.39723958 -3.39723958 -3.39723958]
 [-0.04505712 -0.04505712 -0.04505712 -0.04505712 -0.04505712]]
[array([[-0.15235355, -0.15235355, -0.15235355, -0.15235355, -0.15235355],
       [-3.17578605, -3.17578605, -3.17578605, -3.17578605, -3.17578605],
       [-2.07655263, -2.07655263, -2.07655263, -2.07655263, -2.07655263],
       [-3.39723958, -3.39723958, -3.39723958, -3.39723958, -3.39723958],
       [-0.04505712, -0.04505712, -0.04505712, -0.04505712, -0.04505712]])]


In [41]:
print("=== Test 1: Single-Layer Network Gradient with multi column input ===")

dim_layer = np.array([4, 5])   # 4 inputs -> 5 outputs
testNet = FNN(dim_layer, identity, method_ini="RandomNor")

inputTestVectt = np.ones((4, 1))
inputTestArray = np.concatenate((inputTestVectt,inputTestVectt), axis = 1) 
targetVectt = 0.5 * np.ones((5, 1))
targetTestArray = np.concatenate((targetVectt,targetVectt), axis = 1) 
out = testNet.forward(inputTestArray)
grad = testNet.backward(out[0],out[1],targetTestArray,MeanSquaredErrorDerivative)

#grads = backwards(testNet, inputTestVect, targetVect, MeanSquaredErrorDerivative)

print("\nGradient matrix for single layer:")
print(grad[0])
print(grad)

=== Test 1: Single-Layer Network Gradient with multi column input ===

Gradient matrix for single layer:
[[ 1.79936994  1.79936994  1.79936994  1.79936994  1.79936994]
 [ 0.21423119  0.21423119  0.21423119  0.21423119  0.21423119]
 [-0.95300069 -0.95300069 -0.95300069 -0.95300069 -0.95300069]
 [ 6.1250411   6.1250411   6.1250411   6.1250411   6.1250411 ]
 [-3.84456618 -3.84456618 -3.84456618 -3.84456618 -3.84456618]]
[array([[ 1.79936994,  1.79936994,  1.79936994,  1.79936994,  1.79936994],
       [ 0.21423119,  0.21423119,  0.21423119,  0.21423119,  0.21423119],
       [-0.95300069, -0.95300069, -0.95300069, -0.95300069, -0.95300069],
       [ 6.1250411 ,  6.1250411 ,  6.1250411 ,  6.1250411 ,  6.1250411 ],
       [-3.84456618, -3.84456618, -3.84456618, -3.84456618, -3.84456618]])]


In [ ]:
print("=== Test 2: Deep 10-Layer Network ===")

dim_layer = np.array([
    5, 8, 6, 7, 5, 9, 4, 3, 2, 4, 1
])

deepNet = FNN(dim_layer, identity, method_ini="RandomNor")

inputVect = np.ones((5, 1))
targetVect = np.array([[0.7]])

out = deepNet.forward(inputVect)
grad = deepNet.backward(out[0],out[1],targetVect,MeanSquaredErrorDerivative)

#grads = backwards(deepNet, inputVect, targetVect, MeanSquaredErrorDerivative)

for i, g in enumerate(grad):
    print(f"\nGradient matrix for layer {i}:")
    print(g)


=== Test 2: Deep 10-Layer Network ===

Gradient matrix for layer 0:
[[ -514.78902737  -514.78902737  -514.78902737  -514.78902737
   -514.78902737  -514.78902737]
 [ -676.71823374  -676.71823374  -676.71823374  -676.71823374
   -676.71823374  -676.71823374]
 [  346.65746142   346.65746142   346.65746142   346.65746142
    346.65746142   346.65746142]
 [-1040.04318781 -1040.04318781 -1040.04318781 -1040.04318781
  -1040.04318781 -1040.04318781]
 [  130.90359283   130.90359283   130.90359283   130.90359283
    130.90359283   130.90359283]
 [  400.38919825   400.38919825   400.38919825   400.38919825
    400.38919825   400.38919825]
 [   87.08531237    87.08531237    87.08531237    87.08531237
     87.08531237    87.08531237]
 [ -479.38029786  -479.38029786  -479.38029786  -479.38029786
   -479.38029786  -479.38029786]]

Gradient matrix for layer 1:
[[  513.47164769   209.47718475  1861.60982439   714.93755765
   -499.13420777  1665.30367911  2693.1716353    236.76925538
  -1453.91920696]

In [43]:
print("=== Test 2: Deep 10-Layer Network with multi column input ===")

dim_layer = np.array([
    5, 8, 6, 7, 5, 9, 4, 3, 2, 4, 1
])

deepNet = FNN(dim_layer, identity, method_ini="RandomNor")

inputTestVectt =  np.ones((5, 1))
inputTestArray = np.concatenate((inputTestVectt,inputTestVectt), axis = 1) 
targetVectt = np.array([[0.7]])
targetTestArray = np.concatenate((targetVectt,targetVectt), axis = 1) 

#inputVect = np.ones((5, 1))
#targetVect = np.array([[0.7]])

out = deepNet.forward(inputTestArray)
grad = deepNet.backward(out[0],out[1],targetTestArray,MeanSquaredErrorDerivative)

#grads = backwards(deepNet, inputVect, targetVect, MeanSquaredErrorDerivative)

for i, g in enumerate(grad):
    print(f"\nGradient matrix for layer {i}:")
    print(g)

=== Test 2: Deep 10-Layer Network with multi column input ===

Gradient matrix for layer 0:
[[  43667.95536821   43667.95536821   43667.95536821   43667.95536821
    43667.95536821   43667.95536821]
 [  74996.24529274   74996.24529274   74996.24529274   74996.24529274
    74996.24529274   74996.24529274]
 [-198554.26674218 -198554.26674218 -198554.26674218 -198554.26674218
  -198554.26674218 -198554.26674218]
 [  12034.98456484   12034.98456484   12034.98456484   12034.98456484
    12034.98456484   12034.98456484]
 [  -2183.95514835   -2183.95514835   -2183.95514835   -2183.95514835
    -2183.95514835   -2183.95514835]
 [ -74611.15363515  -74611.15363515  -74611.15363515  -74611.15363515
   -74611.15363515  -74611.15363515]
 [-121797.58105183 -121797.58105183 -121797.58105183 -121797.58105183
  -121797.58105183 -121797.58105183]
 [ -38752.41026223  -38752.41026223  -38752.41026223  -38752.41026223
   -38752.41026223  -38752.41026223]]

Gradient matrix for layer 1:
[[  27113.62921547  -